<a href="https://colab.research.google.com/github/angelinawong1210/VietnameseFakeNews/blob/main/FakeNews_SVM_Sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
truenews = pd.read_csv('/content/Fake.csv')
fakenews = pd.read_csv('/content/Real.csv')

In [ ]:
truenews['True/Fake']='True'
fakenews['True/Fake']='Fake'

In [ ]:
# Combine the 2 DataFrames into a single data frame
news = pd.concat([truenews, fakenews])
news["Article"] = news["tittle"] + news["text"]
news.sample(frac = 1) #Shuffle 100%

,tittle,text,domain,topic,True/Fake,Article
44,Chủ đầu tư nói gì về chuyện BOT An Sương - An ...,Vụ BOT An Sương - An Lạc “thu phí quá hạn”: C...,thanhnien.vn,Giao thông,Fake,Chủ đầu tư nói gì về chuyện BOT An Sương - An ...
56,Tại sao Apple chưa bao giờ quảng cáo trên mạng...,Tài khoản chính thức của Apple có hơn 11 triệ...,sohoa.vnexpress.net,Kinh tế,Fake,Tại sao Apple chưa bao giờ quảng cáo trên mạng...
78,Sẽ bỏ quản lý dân cư bằng hộ khẩu,Chính phủ vừa ra Nghị quyết 112 về việc đơn g...,vnexpress.net,An ninh,Fake,Sẽ bỏ quản lý dân cư bằng hộ khẩu Chính phủ vừ...
48,"Ôtô ở Sài Gòn cuốn hàng loạt người, xe máy vào...",Hiện trường tai nạn. Ảnh: Quế Biên.\n Tối 5/1...,vnexpress.net,Giao thông,Fake,"Ôtô ở Sài Gòn cuốn hàng loạt người, xe máy vào..."
138,Chân dung yêu râu xanh nhẫn tâm sát hại thiếu ...,Chưa dừng lại cơn điên cuồng của một yêu râu x...,NaN,Đời sống,True,Chân dung yêu râu xanh nhẫn tâm sát hại thiếu ...
...,...,...,...,...,...,...
8,"Kinh hoàng bé gái bị chó nhà tấn công, cắn vào...","Chiều 11.9, tại buổi giao ban báo chí Thành ủ...",tintucqpvn.net,Đời sống,True,"Kinh hoàng bé gái bị chó nhà tấn công, cắn vào..."
105,Nghiên cứu: Tóc bạc có thể đen trở lại,Tóc bạc là một trong những dấu hiệu lão hóa. K...,trithucvn.org,Y tế,True,Nghiên cứu: Tóc bạc có thể đen trở lạiTóc bạc ...
109,Cái kết đắng lòng của cô gái chê chàng trai nghèo,Một chàng trai nghèo đã phải lòng con gái của ...,danviet.vn,Đời sống,True,Cái kết đắng lòng của cô gái chê chàng trai ng...
137,Nữ sinh một mình cân 6 thanh niên suốt đêm tro...,Thông qua mạng xã hội Facebook và bạn bè bé gá...,NaN,Đời sống,True,Nữ sinh một mình cân 6 thanh niên suốt đêm tro...


In [ ]:
import nltk 
nltk.download('stopwords')
#Data Cleaning
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def process_text(s):

    # Check string to see if they are a punctuation
    nopunc = [char for char in s if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in nopunc.split()]
    return clean_string

In [ ]:
# Tokenize the Article
#rerun, takes LOOOONG
news['Clean Text'] = news['Article'].apply(process_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=process_text).fit(news['Clean Text'])

print(len(bow_transformer.vocabulary_)) #Total vocab words


298


In [ ]:
#Bag-of-Words (bow) transform the entire DataFrame of text
news_bow = bow_transformer.transform(news['Clean Text'])

In [ ]:
print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

Shape of Sparse Matrix:  (299, 298)
Amount of Non-Zero occurences:  299


In [ ]:
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(299, 298)


In [ ]:
#Train SVM model
from sklearn.svm import SVC
fakenews_detect_model = SVC(kernel = 'sigmoid', random_state = 0).fit(news_tfidf, news['True/Fake'])


In [ ]:
#Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))


              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       150
        True       1.00      1.00      1.00       149

    accuracy                           1.00       299
   macro avg       1.00      1.00      1.00       299
weighted avg       1.00      1.00      1.00       299



In [ ]:
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(news['Article'], news['True/Fake'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

209 90 299


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  
    ('tfidf', TfidfTransformer()),  
    ('classifier', SVC(kernel = 'sigmoid', random_state = 0)),  
])
pipeline.fit(news_train,text_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x7f510b9a2160>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', SVC(kernel='sigmoid', random_state=0))])

In [ ]:
prediction = pipeline.predict(news_test)

In [ ]:
print(classification_report(prediction,text_test))

              precision    recall  f1-score   support

        Fake       0.85      0.89      0.87        46
        True       0.88      0.84      0.86        44

    accuracy                           0.87        90
   macro avg       0.87      0.87      0.87        90
weighted avg       0.87      0.87      0.87        90

